In [1]:
import multiprocessing
import gensim
from gensim.models.word2vec import Word2Vec
import nltk
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

c:\programdata\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Taller 03: Análisis de Sentimiento de Reviews de Guía Oleo

## Alumnos: 
- Alvarez Muñoz Carolina
- Escobar Aldo 

## Objetivo

El objetivo de este taller es doble. En primer lugar que estudien por su cuenta qué es lo que hacen los diferentes parámetros de word2vec en gensim, y en segundo lugar que usen word2vec y experimenten un poco con sus resultados.

## Parte 1

Lean la sección de word2vec del capítulo 6 de la última edición del libro de Jurafsky y Martin (https://web.stanford.edu/~jurafsky/slp3/6.pdf) y también busquén posts online referidos a word2vec (por ej. http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/, http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/, https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial). Una vez que hayan hecho esto respondan en no más de tres oraciones qué rol cumple cada uno de estos parámetros.

* **min_count**: Es un integer que funciona como un valor "techo" con el fin de que el modelo ignore todas las palabras con una frecuencia inferior a este valor. Las palabras extremadamente infrecuentes generalmente no son importantes, por lo que es mejor deshacerse de ellas, a menos que el data set sea realmente pequeño, esto no afectaría al modelo.
* **window**: Es un integer que determina cuántas palabras antes y después de un String dado se incluirían como palabras de contexto del String en cuestión. Valores de ventana chicos dan resultados de naturaleza más sintáctica, en cambio valores más grandes dan resultados de naturaleza más semántica. Este último lleva más tiempo de training dado que el tamaño de la ventana es más grande.
* **size**: Es un integer que representa la dimensionalidad del vector. Cuanto mayor sea el número de dimensiones que está utilizando, más información se puede almacenar. Los modelos vectoriales densos tienen una dimensionalidad de 50-300 y las dimensiones son más difíciles de interpretar.
* **sample**: Es un float que representa el umbral que permite configurar las palabras con mayor frecuencia en los datos de entrenamiento para que sean muestreados aleatoriamente. El valor predeterminado es 1e-3, el rango útil es (0, 1e-5) (Ver: https://github.com/RaRe-Technologies/gensim/blob/727caf220ba2ebe253beaee73877afbd99b2db02/gensim/models/word2vec.py#L600)
* **negative**: Es un integer que representa el número de ejemplos negativos (muestras de palabras que no son vecinas), por defecto su valor es 5 y los valores comunes son entre 3 a 10. Si su valor es 0, no se utiliza muestreo negativo. Esta idea de muestreo negativo está inspirada en este [paper](http://proceedings.mlr.press/v9/gutmann10a/gutmann10a.pdf) sobre estimación de contraste de ruido, por lo que estamos constrastando la señal real (ejemplos positivos de palabras vecinas) con ruido (palabras seleccionadas al azar que no son vecinas), lo que permite [una gran compensación de eficiencia computacional y estadística](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling).  La idea del muestreo negativo es que si el modelo puede distinguir entre los pares probables (positivos) frente a los pares improbables (negativos), se aprenderán buenos vectores de palabras.
* **ns_exponent**: Es un float que representa el exponente utilizado para dar forma a la distribución de muestreo negativa. Un valor de 1.0 muestrea exactamente en proporción a las frecuencias, 0.0 muestrea todas las palabras por igual, mientras que un valor negativo muestrea más palabras de baja frecuencia que palabras de alta frecuencia. El popular valor predeterminado de 0.75 fue elegido por el paper original de Word2Vec. Algunas investigaciones sugieren que otras aplicaciones, como los sistemas de recomendación, podrían beneficiarse de otros valores.

## Parte 2

En esta parte pueden seguir dos caminos:

1. Bajar 50000 notas de La Nación elegidas al azar (puede reutilizar las ~11000 entregadas en clase).
2. Utilizar un corpus de su elección (puede ser alguno que se encuentre disponible online o alguno que ustedes hayan creado -por ejemplo texto en foros de Rava Online, Artículos de Wikipedea-).

Utilizando el corpus elegido, entrenen dos modelos de word2vec, uno con una ventana pequeña y otro con una ventana grande (qué es grande y qué es pequeña queda a su criterio). Una vez entrenado evalue si los vectores encontrados tienen sentido (si los mismos captan bien similitud entre palabras y si logran captar analogías).

Para resolver este punto no deben entregar el código que **descarga** el corpus, si no que el código debe comenzar suponiendo que ya tienen el corpus a analizar en sus máquinas.

# RESOLUCIÓN

## Lectura y preparación del corpus de Quora Questions.
Preprocesamos el texto y filtramos los datos, quedandonos con aquellos que no están duplicados.

In [2]:
def read_questions(row,column_name):
    return gensim.utils.simple_preprocess(str(row[column_name]).encode('utf-8'))

In [3]:
df = pd.read_csv("./dataQuora.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
len(df)

404290

In [ ]:
documents = []
for index, row in df.iterrows():
    documents.append(read_questions(row,"question1"))
    if row["is_duplicate"] == 0:
        documents.append(read_questions(row,"question2"))

In [ ]:
documents[5]

## Modelo 1 (WINDOW = 1)

En este modelo intentaremos probar window=1. Efectivamente, como mencionamos en su definición, se ve claramente que el modelo logra "aprender" sintácticamente el corpus sobre el cual fue entrenado. 

En ambos modelos se entreno con sg=1 (Skip-Gram) ya que brindó mejores resultados que con sg=0.


In [ ]:
# Defino los parámetros del modelo
model1 = gensim.models.Word2Vec(documents, size=100,
                               window=1,
                               min_count=1,
                               workers=multiprocessing.cpu_count(), sg=1)

In [ ]:
print('Model vocab size : ', len(model1.wv.vocab))

### Similitud entre 2 palabras
Chequeamos que el primer modelo detecta alta similitud entre django y flask, entre scala y python y entre csv y json, al igual que el modelo 2. Pero para los pares (phonegap, mobile) el modelo no detecta mucha similitud, con respecto al modelo 2. Pasa al revés con el par (holden,spark)  

In [ ]:
model1.wv.similarity('django', 'flask')

In [ ]:
model1.wv.similarity('scala', 'python')

In [ ]:
model1.wv.similarity("csv", "json")

In [ ]:
model1.wv.similarity("phonegap", 'mobile')

In [ ]:
model1.wv.similarity("holden", "spark")

### Palabras mas similares
Con respecto al modelo 2, difiere mucho en udemy, testing y phonegap.
- El modelo 2, para phonegap, identificó muy bien a xamarin como tecnología alternativa. Tambien sdk (termino muy relacionado  a mobile) y prototyping (uno de sus principales usos)
- El modelo 2 para udemy capto mas plataformas educativas online que el modelo 1.
- El modelo 1 para testing capta mejor ciertos procesos, etapas y actividades de la Ingenieria del Software, en cambio el modelo 2 capta mejor ciertas tecnologías o terminologías relacionadas a la palabra 'testing'

Con el resto de los términos, ambos modelos capturan más o menos lo mismo.

In [ ]:
model1.wv.most_similar(positive=['json', 'python'], negative=['java'])

In [ ]:
model1.wv.most_similar(positive=['vpn', 'python'], negative=['java'])

In [ ]:
model1.most_similar('facebook', topn=10)

In [ ]:
model1.wv.most_similar('udemy')

In [ ]:
model1.wv.most_similar('testing')

In [ ]:
model1.wv.most_similar('phonegap')

### Palabra que no matchea en la lista
El modelo 1 detecta bien, al igual que el modelo 2, aquellas palabras que no "matchean con el resto".

In [ ]:
model1.wv.doesnt_match(['python', 'sql', 'javascript', 'java'])

In [ ]:
model1.wv.doesnt_match(['python', 'vpn', 'javascript', 'java'])

In [ ]:
model1.wv.doesnt_match(['beer', 'coffee', 'coding'])

## SEGUNDO MODELO: (WINDOW = 4)

In [ ]:
# Defino los parámetros del modelo
model2 = gensim.models.Word2Vec(documents, size=100,
                               window=4,
                               min_count=1,
                               workers=multiprocessing.cpu_count(), sg=1)

In [ ]:
print('Model vocab size : ', len(model2.wv.vocab))

### Similitud entre 2 palabras

In [ ]:
model2.wv.similarity('django', 'flask')

In [ ]:
model2.wv.similarity('scala', 'python')

In [ ]:
model2.wv.similarity("csv", "json")

In [ ]:
model2.wv.similarity("phonegap", 'mobile')

In [ ]:
model2.wv.similarity("holden", "spark")

### Palabras mas similares

In [ ]:
model2.wv.most_similar(positive=['json', 'python'], negative=['java'])

In [ ]:
model2.wv.most_similar(positive=['vpn', 'python'], negative=['java'])

In [ ]:
model2.most_similar('facebook', topn=10)

In [ ]:
model2.wv.most_similar('udemy')

In [ ]:
model2.wv.most_similar('testing')

In [ ]:
model2.wv.most_similar('phonegap')

### Palabra que no matchea en la lista
El modelo 1 detecta bien, al igual que el modelo 2, aquellas palabras que no "matchean con el resto".

In [ ]:
model2.wv.doesnt_match(['python', 'sql', 'javascript', 'java'])

In [ ]:
model2.wv.doesnt_match(['python', 'vpn', 'javascript', 'java'])

In [ ]:
model2.wv.doesnt_match(['beer', 'coffee', 'coding'])

## Modalidad de entrega

El taller puede ser resuelto en grupos de a lo sumo 3 integrantes.

La resolución consiste en escribir una notebook que resuelva lo que la consigna pide.

Se evaluará que el código entregado sea claro y legible.

Si el código no corre, el taller no estará aprobado y deberá ser reentregado.

La entrega debe hacerse a rgalvez@utdt.edu.